In [1]:
import pandas as pd

import re
import itertools
from collections import Counter
from typing import  Dict, List, Tuple

from tqdm import tqdm
from transformers import AutoTokenizer

2021-10-01 13:16:37.767988: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
Train_csv_file = '/opt/ml/dataset/train/train.csv'
Test_csv_file = '/opt/ml/dataset/test/test_data.csv'

df = pd.read_csv(Train_csv_file)
df = df.drop_duplicates(['sentence','subject_entity','object_entity','label']).reset_index()

In [3]:
MODEL_NAME = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

vocab = list(tokenizer.get_vocab().keys())
unused_list = [word for word in vocab if word.startswith('[unused')]        
print(f'unused token count: {len(unused_list)}')

/opt/conda/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


unused token count: 500


# 🛰️ 전처리

In [26]:
def remove_special_char(sentence):
    """ 특수문자 및 독일어 제거, 수정"""
    sentence = re.sub(r'[À-ÿ]+','', sentence) # 독일어
    sentence = re.sub(r'[\u0600-\u06FF]+','', sentence)  # 사우디어
    sentence = re.sub(r'[ß↔Ⓐب€☎☏±]+','', sentence)
    sentence = re.sub('–','─', sentence)
    sentence = re.sub('⟪','《', sentence)
    sentence = re.sub('⟫','》', sentence)
    sentence = re.sub('･','・', sentence)
    return sentence

test_sen = 'Hermann Müller'
remove_special_char(test_sen)

TypeError: expected string or bytes-like object

In [5]:
def add_space_char(sentence) :
    def add_space(match) :
        res_str = ', '.join(match.group().split(',')).rstrip()
        return res_str
    p = re.compile(r'([기-힣\w\-]+,)+[기-힣\w\-]+')
    sentence = p.sub(add_space, sentence)
    return sentence
test_sen = '앨범에는 에미넴,G-Unit,닥터드레,제이미 폭스 등이 참여하고,영국차트에서도 1위를 한다.'
add_space_char(test_sen)

'앨범에는 에미넴, G-Unit, 닥터드레, 제이미 폭스 등이 참여하고, 영국차트에서도 1위를 한다.'

In [6]:
def substitution_date(sentence):
    """
    기간 표시 '-' => '~'
    1223년 – => 1223년 ~ 
    """
    def sub_tibble(match) :
        res_str = re.sub('[–\-]','~',match.group())
        return res_str
    re_patterns = [
        r'(\d{2,4}년\s*)(\d{1,2}[월|일]\s*)(\d{1,2}[월|일])\s*[–\-]',
        r'(\d{2,4}년\s*)(\d{1,2}[월|일]\s*)\s*[–\-]',
        r'(\d{2,4}년\s*)\s*[–\-]',
        r'\((\d{4}[–\-]\d{2,4})\)'
    ]
    for re_pattern in re_patterns :
        p = re.compile(re_pattern)
        sentence = p.sub(sub_tibble, sentence)   
    return sentence

test_sen = '후 시니어 대회로 올라가서 (1934–1942) 시즌 ISU 쇼트트랙 월드컵에 4차례 출전하여 금메달 4개, 은메달 5개를 차지하였으며, 2013-14 시즌에서 김아랑은 쇼트트랙 스피드'
substitution_date(test_sen)

'후 시니어 대회로 올라가서 (1934~1942) 시즌 ISU 쇼트트랙 월드컵에 4차례 출전하여 금메달 4개, 은메달 5개를 차지하였으며, 2013-14 시즌에서 김아랑은 쇼트트랙 스피드'

In [7]:
def add_space_year(sentence):
    """
    숫자와 년 사이에 공백
    1223년 => 1223 년 => ⑦ 년
    """
    def add_space(match) :
        # res_str = '⑦ ' + match.group()[4:]
        res_str =  match.group()[:4] +' ' + match.group()[4:]
        return res_str
    p = re.compile(r'\d{4}년')
    sentence = p.sub(add_space, sentence)
    return sentence
test_sen = '2010년에는 아시아 가수 최초로 마이클 잭슨의 곡을 리메이크하였는데'
add_space_year(test_sen)

'2010 년에는 아시아 가수 최초로 마이클 잭슨의 곡을 리메이크하였는데'

In [8]:
def preprocessing(sentence) :
    sent = remove_special_char(sentence)
    sent = substitution_date(sent)
    # sent = add_space_year(sent)
    sent = add_space_char(sent)
    return sent

In [9]:
df = pd.read_csv(Train_csv_file)

ess = ['sentence','subject_entity','object_entity']
preprocessed_df = df.copy()
for col in ess :
    preprocessed_df[col] = preprocessed_df[col].apply(preprocessing)

# 🛰️UNK으로 변하는 word 및 char 확인

In [10]:
from IPython.core.display import HTML
def word_highligt_html(txt, word, color='black', highlight=None, attr=None):
    if isinstance(word, str):
        txt = txt.replace(word, f'<span style="color: {color}; background-color:{highlight}">{word}</span>')
    else:
        if not isinstance(color, list):
            color = [color] * len(word)
        if not isinstance(highlight, list):
            highlight = [highlight] * len(word)
        for w, c, h in zip(word, color, highlight):
            txt = txt.replace(w, f'<span style="color: {c}; background-color:{h}">{w}</span>')
    return txt

In [16]:
def subword_parsing(wordpiece:List) -> List[str]: ## subword # 제거용
    Known_char = []
    for subword in wordpiece :
        if subword == tokenizer.unk_token :
            Known_char.append(tokenizer.unk_token)
        else :
            string = subword.replace('#','')
            Known_char.extend(string)
    return Known_char


def UNK_word_and_chr(text:str) -> Tuple[List[str], List[str]]:
    sub_word_UNK_list = []
    word_UNK_list = []
    
    def add_space(match) :
        bracket = match.group()
        added = ' ' + bracket + ' '
        return added
    p = re.compile(r'[\([)\]|,|-|~|-|‘|’]')
    words_list = p.sub(add_space, text).split()
    for word in words_list :
        subwordpieces_ID_encoded = tokenizer.tokenize(word)
        Known_subword = subword_parsing(subwordpieces_ID_encoded)
        # print(word, len(word), Known_subword, len(Known_subword))
        for sub_char, NK_char in zip(word, Known_subword) :
            if sub_char != NK_char and len(word) == len(Known_subword) :
                sub_word_UNK_list.append(sub_char)
            elif sub_char != NK_char and len(word) != len(Known_subword) :
                sub_word_UNK_list.append(word)
                break
            elif len(word) == len(Known_subword) :
                pass
            else :
                print('else', word, len(word), Known_subword, len(Known_subword))
                break
    return sub_word_UNK_list
    
text ='박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다.'
print(UNK_word_and_chr(text))
if tokenizer.unk_token in tokenizer.tokenize(text) :
    print(tokenizer.tokenize(text))

['容', '旿']
['박용', '##오', '(', '朴', '[UNK]', '[UNK]', ',', '1937', '##년', '4', '##월', '29', '##일', '(', '음력', '3', '##월', '19', '##일', ')', '(', '음력', '3', '##월', '19', '##일', ')', '~', '2009', '##년', '11', '##월', '4', '##일', ')', '는', '서울', '##에서', '태어난', '대한민국', '##의', '기업인', '##으로', '두산', '##그룹', '회장', ',', 'KBO', '총재', '등', '##을', '역임', '##했', '##다', '.']


# 🛰️ UNK 분포

In [17]:
def UNK_word(df_col:pd.DataFrame, unk_idx:int) -> List[Tuple] :
    UNK_word_idx = []
    for i, word in enumerate(tqdm(df_col)) :
        if unk_idx in tokenizer.encode(word) :
            UNK_word_idx.append((i,word))
    return UNK_word_idx

unk_idx = tokenizer.get_vocab()[tokenizer.unk_token] # 3

sub_col = preprocessed_df['subject_entity'].apply(eval).str['word']
sub_UNK = UNK_word(sub_col, unk_idx)

obj_col = preprocessed_df['object_entity'].apply(eval).str['word']
obj_UNK = UNK_word(obj_col, unk_idx)

sen_col = preprocessed_df['sentence']
sen_UNK = UNK_word(sen_col, unk_idx)

print(len(sub_UNK), len(obj_UNK), len(sen_UNK))
print(sub_UNK[0]) # (preprocessed_df row index, entity)
print(obj_UNK[0]) # (preprocessed_df row index, entity)
print(sen_UNK[0]) # (preprocessed_df row index, senetence)

100%|██████████| 32470/32470 [00:08<00:00, 3979.65it/s]

222 259 2965
(21, '스티브 바라캇')
(191, '少弐氏')
(7, '박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다.')


In [18]:
txt = ''
count = 1
unk_len = 0
for idx, sen in tqdm(sen_UNK) :
    UNK_subword = UNK_word_and_chr(sen)
    txt += word_highligt_html(sen, UNK_subword, ['white']*len(UNK_subword),  ['#96C4ED']*len(UNK_subword)) + '<br/><br/>'
    if count > 5:
        break
    count += 1
HTML(txt)

  0%|          | 5/2965 [00:00<00:07, 370.45it/s]


## 🛰️ Sentece UNK 확인

In [19]:
cnt = 1
UNK_sentence_list = []
for sen in tqdm(preprocessed_df['sentence']) :
    if tokenizer.unk_token in tokenizer.tokenize(sen) :
        UNK_sentence_list.extend(UNK_word_and_chr(sen))
        cnt+=1
print(cnt)

  3%|▎         | 884/32470 [00:00<00:14, 2218.91it/s]

else 空海くうかい홍법 8 ['空', '海', '[UNK]'] 3
else "촐로" 4 ['"', '[UNK]', '"'] 3


  7%|▋         | 2218/32470 [00:01<00:13, 2205.33it/s]

else '숲튽훈'이라는 8 ["'", '[UNK]', "'", '이', '라', '는'] 6
else '숲튽훈 4 ["'", '[UNK]'] 2


 10%|█         | 3318/32470 [00:01<00:13, 2143.92it/s]

...

else 0.2µT 5 ['0', '.', '[UNK]'] 3
else 0.3µT 5 ['0', '.', '[UNK]'] 3


 99%|█████████▉| 32137/32470 [00:14<00:00, 2233.28it/s]

else '촐라 3 ["'", '[UNK]'] 2
else "챠미 3 ['"', '[UNK]'] 2


100%|██████████| 32470/32470 [00:14<00:00, 2204.39it/s]

2966


In [20]:
for idx, cont in enumerate(Counter(UNK_sentence_list).most_common(100)) :
    if idx % 10 == 9 :
        print()
    else :
        print(cont, end="\t")

('李', 225)	('崔', 60)	('皇', 60)	('后', 54)	('永', 41)	('尹', 38)	('昌', 33)	('慶', 30)	('俊', 29)	
('趙', 25)	('興', 24)	('홋스퍼', 24)	('孝', 23)	('盧', 22)	('承', 22)	('梁', 22)	('容', 21)	('徐', 21)	
('熙', 21)	('貞', 20)	('沈', 20)	('陵', 19)	('鍾', 19)	('錫', 18)	('放', 18)	('池', 18)	('團', 18)	
('賢', 18)	('洪', 18)	('申', 17)	('進', 17)	('洙', 17)	('泰', 17)	('植', 16)	('夏', 16)	('秀', 16)	
('校', 16)	('勳', 16)	('吳', 16)	('康', 15)	('景', 15)	('홋카이도', 15)	('炳', 15)	('恩', 15)	('哲', 14)	
('羅', 14)	('源', 14)	('惠', 14)	('範', 14)	('榮', 14)	('煥', 14)	('宇', 14)	('崇', 14)	('少', 13)	
('忠', 13)	('姬', 13)	('숀', 13)	('浩', 12)	('嬪', 12)	('根', 12)	('唐', 12)	('翁', 12)	('鉉', 12)	
('建', 12)	('桓', 12)	('玉', 11)	('敬', 11)	('淑', 11)	('恭', 11)	('智', 11)	('宣', 11)	('許', 11)	
('樂', 10)	('延', 10)	('昭', 10)	('順', 10)	('奎', 10)	('斗', 10)	('應', 10)	('鎬', 10)	('베렝가리오', 10)	
('촐라', 10)	('奉', 10)	('藤', 10)	('澤', 10)	('閔', 10)	('織', 10)	('弐', 9)	('泳', 9)	('寧', 9)	


In [21]:
cnt = 1
for sen in preprocessed_df['sentence'] :
    unk_char = "".join(UNK_word_and_chr(sen))
    if re.search(r'[^\u4e00-\u9fff|\u3131-\u3163\uac00-\ud7a3]+', unk_char) :
        print(UNK_word_and_chr(sen),sen)
        cnt += 1
    if cnt > 10 :
        break
print(cnt)

['Трбайх'] 구시 칸(1582년 ~ 1655년 1월 14일)은 별명 그시리 칸, 본명 토르바이크(Трбайх, Torbaikh Khan)는 17세기 중반 몽골 오이라트부의 네 씨족 중 하나인 코슈트의 귀족으로, 코슈트 칸국의 창업군주다.
else 空海くうかい홍법 8 ['空', '海', '[UNK]'] 3
['Κυριάκος', 'Παπαδόπουλος'] 키리아코스 파파도풀로스 (Κυριάκος Παπαδόπουλος, Kyriakos Papadopoulos, 1992년 2월 23일, 중앙마케도니아 주 카테리니 ~)는 그리스의 축구 선수이다.
else "촐로" 4 ['"', '[UNK]', '"'] 3
['Ζ'] 12세기 요한네스 카라테로스의《점성술 개론(Compendium of Astrology)》에 있는 한 도해는 태양이 하나의 광선을 가진 원으로 표현되어 있고, 목성은 (그리스 신화에서 유피테르에 상응하는 제우스의 첫글자인) "제타"(Ζ)로 표시되어 있으며, 화성은 창이 겹쳐진 방패로 그려져 있다.
['체르노젬스키', 'Величко', 'Димитров', 'Керин', 'Величко', 'Димитров', 'Керин'] 블라도 체르노젬스키(본명: 벨리치코 디미트로프 케린(Величко Димитров Керин)(Величко Димитров Керин), 1897년 10월 19일 ~ 1934년 10월 9일)는 불가리아의 혁명가이다.
['MŠK'] 이에 그는 2002년 여름 자국 리그의 MŠK 질리나로 이적을 선택하였고, 그해 슬로바키아 수페르리가 챔피언십에서 소속팀이 두 번째 우승컵을 들어올리는 데 결정적인 공헌을 하였다.
['Парламентские', 'выборы', 'в', 'России'] 2011년 러시아 의회 선거(Парламентские выборы в России 2011)는 2011년 450석의 국가두마 의회 의원들을 뽑기 위한 총선거이다.
['뗀다.'] 대표팀은 한국시간으로 10일 오후 11시 투르크메니스탄의 수도 

In [22]:
cnt = 1
for sen in preprocessed_df['sentence'] :
    unk_char = "".join(UNK_word_and_chr(sen))
    if re.search(r'[\u0600-\u06FF]+', unk_char) :
        print(UNK_word_and_chr(sen),sen)
        cnt += 1
    if cnt > 10 :
        break
print(cnt)

else 空海くうかい홍법 8 ['空', '海', '[UNK]'] 3
else "촐로" 4 ['"', '[UNK]', '"'] 3
else '숲튽훈'이라는 8 ["'", '[UNK]', "'", '이', '라', '는'] 6
else '숲튽훈 4 ["'", '[UNK]'] 2
else '펭귄의 4 ["'", '[UNK]'] 2
else 株式会社アニメーションドゥウ 14 ['株', '式', '[UNK]', '社', '[UNK]'] 5
else '캥거루'로 6 ["'", '[UNK]', "'", '로'] 4
else "Auseklītis" 12 ['"', '[UNK]', '"'] 3
else "Ausekliņis" 12 ['"', '[UNK]', '"'] 3
else "Ausekleņis" 12 ['"', '[UNK]', '"'] 3
else "Ausekliņa" 11 ['"', '[UNK]', '"'] 3
else "Auseklītis" 12 ['"', '[UNK]', '"'] 3
else "호엔촐레른베링겐"을 11 ['"', '[UNK]', '"', '을'] 4
else '쌉니다 4 ["'", '[UNK]'] 2
else 民主・リベラル労組会議 11 ['民', '主', '・', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '議'] 8
else 합병하면서「홋카이도 10 ['합', '병', '하', '면', '서', '「', '[UNK]'] 7
else '크리스챤 5 ["'", '[UNK]'] 2
else "호엔촐레른베링겐"을 11 ['"', '[UNK]', '"', '을'] 4
else /ˈkeɪ.ɒs/ 9 ['/', '[UNK]', '.', '[UNK]', '/'] 5
else 《恋のサイン 6 ['《', '[UNK]', '[UNK]'] 3
else "호엔촐레른베링겐"을 11 ['"', '[UNK]', '"', '을'] 4
else "쉘부르"라는 7 ['"', '[UNK]', '"', '라', '는'] 5
else "Aḥmad 6 ['"', '[U

## 🛰️ entity UNK 확인

In [23]:
subject_entity = []
object_entity = []

for i, j in zip(preprocessed_df['subject_entity'], preprocessed_df['object_entity']):
    i = eval(i)['word']
    j = eval(j)['word']

    subject_entity.append(i)
    object_entity.append(j)

In [24]:
cnt = 1
UNK_entity_list = []
for sen in tqdm(subject_entity+object_entity) :
    # if '李' in sen :
    if tokenizer.unk_token in tokenizer.tokenize(sen) :
        # print(sen, tokenizer.tokenize(sen))
        # print(UNK_word_and_chr(sen))
        UNK_entity_list.extend(UNK_word_and_chr(sen))
        cnt += 1
print(cnt)

 59%|█████▉    | 38543/64940 [00:03<00:02, 12240.11it/s]

else 株式会社アニメーションドゥウ 14 ['株', '式', '[UNK]', '社', '[UNK]'] 5


 89%|████████▉ | 58097/64940 [00:04<00:00, 12156.87it/s]

else 中川亜紀子なかがわ 9 ['中', '川', '[UNK]', '[UNK]', '子', '[UNK]'] 6


100%|██████████| 64940/64940 [00:05<00:00, 11986.43it/s]

482


In [25]:
Counter(UNK_entity_list).most_common(100)
for idx, cont in enumerate(Counter(UNK_entity_list).most_common(120)) :
    if idx % 10 == 9 :
        print()
    else :
        print(cont, end="\t")

('홋스퍼', 27)	('홋카이도', 10)	('放', 9)	('送', 9)	('베렝가리오', 8)	('에스파뇰', 8)	('숀', 8)	('織', 8)	('쥘', 7)	
('陵', 7)	('李', 7)	('리콴유', 6)	('쾰른', 6)	('우젠슝', 6)	('리셴녠', 5)	('비욘세', 5)	('少', 5)	('弐', 5)	
('포로셴코', 4)	('묀헨글라트바흐', 4)	('다롄', 4)	('局', 4)	('昌', 4)	('梁', 4)	('啓', 4)	('超', 4)	('弓', 4)	
('흄', 3)	('쟝쩐위', 3)	('뮐러', 3)	('훙윈', 3)	('푸르트벵글러', 3)	('스뱌토폴크', 3)	('슌지', 3)	('와치랄롱꼰', 3)	('아녜스', 3)	
('藤', 3)	('晋', 3)	('聯', 3)	('香', 3)	('바라캇', 2)	('넋업샨', 2)	('루쳄부르스키', 2)	('꽈드로스', 2)	('래이쾨넨', 2)	
('뱌체슬라프', 2)	('쥰', 2)	('욘', 2)	('카뮈', 2)	('헴스워스', 2)	('귈', 2)	('슈쳉스니', 2)	('푀', 2)	('힉스', 2)	
('하이츨러', 2)	('젭', 2)	('로바쳅스키', 2)	('昶', 2)	('위쾨넨', 2)	('퓌르트', 2)	('파스콸', 2)	('맬', 2)	('아르툠', 2)	
('똔텃투옛', 2)	('앳킨슨', 2)	('브뢴뷔', 2)	('에미넴', 2)	('핼리팩스', 2)	('總', 2)	('켐니츠', 2)	('伊', 2)	('達', 2)	
('廳', 2)	('樹', 2)	('郭', 2)	('驥', 2)	('居', 2)	('干', 2)	('永', 2)	('衛', 2)	('乃', 2)	
('麗', 2)	('뎀나', 2)	('降', 2)	('内', 2)	('后', 2)	('洋', 2)	('技', 2)	('術', 2)	('洪', 2)	
('興', 2)	('豊', 2)	('鍾', 2)	('煦', 2)	('柳', 2)	('浙', 2)	('港', 2)	('裔', 2

# 🛰️시험

In [90]:
Test_csv_file = '/opt/ml/dataset/test/test_data.csv'
test_df = pd.read_csv(Test_csv_file)

In [92]:
ess = ['sentence','subject_entity','object_entity']
for col in ess :
    test_df[col] = test_df[col].apply(preprocessing)

In [93]:
t_sub_col = test_df['subject_entity'].apply(eval).str['word']
t_sub_UNK = UNK_word(t_sub_col, unk_idx)

t_obj_col = test_df['object_entity'].apply(eval).str['word']
t_obj_UNK = UNK_word(t_obj_col, unk_idx)

t_sen_col = test_df['sentence']
t_sen_UNK = UNK_word(t_sen_col, unk_idx)

100%|██████████| 7765/7765 [00:01<00:00, 4339.03it/s]


In [94]:
cnt = 0
for idx, sen in t_sen_UNK :
    UNK_subword = UNK_word_and_chr(sen)
    print(UNK_subword, sen)
    print(tokenizer.tokenize(sen))
    cnt += 1
print(cnt)

['姜'] 강명구(姜明求, 1917년~2000년)는 대한민국의 건축가이다.
['강', '##명', '##구', '(', '[UNK]', '明', '求', ',', '1917', '##년', '~', '2000', '##년', ')', '는', '대한민국', '##의', '건축가', '##이다', '.']
['栗', '亭', '李', '寬'] 일찍이 이천(利川)의 율정(栗亭) 이관의(李寬義)의 문하에서 수학하였다.
['일찍이', '이천', '(', '利', '川', ')', '의', '율', '##정', '(', '[UNK]', '[UNK]', ')', '이관', '##의', '(', '[UNK]', '[UNK]', '義', ')', '의', '문하', '##에서', '수학', '##하', '##였', '##다', '.']
['코'] 잉글랜드와 로마 사이의 친밀감은 1526년 코냑 동맹의 공식화에서 볼 수 있다.
['잉글랜드', '##와', '로마', '사이', '##의', '친밀감', '##은', '152', '##6', '##년', '[UNK]', '동맹', '##의', '공식', '##화', '##에서', '볼', '수', '있', '##다', '.']
['。'] 초기에는 대부분의 경우 '탈퇴'라는 표현을 사용하였지만(결성 당시부터 나카자와 졸업까지의 모닝구무스메。의 역사가 게재되어 있는 ASAYAN 공식 사이트 등에서는 지금도 탈퇴라는 표현으로 통일되어 있다.) 2001년 이후에는 공식적으로 주로 졸업이라는 표현을 쓰게 되어('졸업 메모리얼' 굿즈 등), 과거 멤버에 대해서도 졸업이라는 표현으로 통일한 경우도 많다(콘서트 MC에서의 '졸업 멤버' 호칭 등).
['초기', '##에', '##는', '대부분', '##의', '경우', "'", '탈퇴', "'", '라는', '표현', '##을', '사용', '##하', '##였', '##지만', '(', '결성', '당시', '##부터', '나카', '##자와', '졸업', '##까', '##지', '##의'

# 🛸 한 단어로 이루어지지 않은 entity
- 여러 단어
- 특정 패턴이 존재하는
- 맞추기 어려울 것 같은
+ 위 내용을 만족하는 entity는 entity token wrap 해주는게 어떠려나?

In [9]:
for obj in df['object_entity'] :
    d_obj = eval(obj)
    if d_obj['type'] == 'DAT' :
        if len(d_obj['word'].split()) > 1 :
            print(obj)
            test_encoced = tokenizer.encode(d_obj['word'])
            print(test_encoced)
            Id_to_tokens = tokenizer.convert_ids_to_tokens(test_encoced[1:-1])
            print(Id_to_tokens)
            break

{'word': '1937년 4월 29일', 'start_idx': 9, 'end_idx': 20, 'type': 'DAT'}
[0, 20533, 2440, 24, 2429, 4346, 2210, 2]
['1937', '##년', '4', '##월', '29', '##일']
